<a href="https://colab.research.google.com/github/Ogweno/portfolio/blob/main/download_google_images_rev1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code uses selenium and beautifulsoup to download and save the images to a folder.

The folder is created if it does not exist.
First we have to install chromium without snap

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF


Executing: /tmp/apt-key-gpghome.p5ozHtyLPQ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.Pkf5mxiWE5/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.Wwymk6sc9k/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


Update and install chromium chromium driver and selenium

In [ ]:
!apt-get update
!apt-get install chromium chromium-driver
!pip3 install selenium

from selenium import webdriver
from selenium.webdriver.chrome.options import Options


Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:6 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:11 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Get:12 http://deb.debian.org/debian buster/main amd64 Packages [10.7 MB]
Get:13 http://archive.ubuntu.com/ubuntu focal-backports 

Load the necessary libraries to be used for downloading and path to where the imagess are to be saved

In [ ]:
from bs4 import BeautifulSoup
import requests
import urllib.request
import os
import time
from selenium.webdriver.common.by import By

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Create directory if it doesn't exist
directory = '/content/drive/MyDrive/Independent_Study_2023/lawn_images'
if not os.path.exists(directory):
    os.makedirs(directory)


search_url = 'https://www.google.com/search?q=lawn+large&tbm=isch&ved=2ahUKEwj8vPSBlZb9AhU5zskDHQeJDEcQ2-cCegQIABAA&oq=lawn+large&gs_lcp=CgNpbWcQAzIGCAAQCBAeMgYIABAIEB4yBggAEAgQHjIGCAAQCBAeMgYIABAIEB4yBggAEAgQHjIGCAAQCBAeMgYIABAIEB4yBggAEAgQHjIGCAAQCBAeOgcIABCxAxBDOggIABCABBCxAzoFCAAQgAQ6BAgAEENQsAVYxA5gnxBoAHAAeACAAW2IAdMEkgEDNi4xmAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=_BjsY_zOB7mcp84Ph5KyuAQ&bih=569&biw=1280'
#url = "http://example.com/"
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome("chromedriver", options=options)
#driver.get(search_url)


num_images = 300 # number of images to be downloaded if available

# Scroll down the page to load more images
scroll_down = "window.scrollTo(0, document.body.scrollHeight);"
SCROLL_PAUSE_TIME = 1.5
last_height = 0

#driver = webdriver.Chrome() #replace with appropriate web driver
driver.get(search_url)

while True:
    driver.execute_script(scroll_down)
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    #if len(driver.find_elements_by_tag_name('img')) >= num_images:
    if len(driver.find_elements(By.TAG_NAME, 'img')) >= num_images:
        break

images = driver.find_elements(By.TAG_NAME, 'img')[:num_images]

for i, img in enumerate(images):
    url = img.get_attribute('src')
    if url is not None and 'http' in url:
        filename = os.path.join(directory, f"lawnimage{i+1}.jpg")
        urllib.request.urlretrieve(url, filename)
        print(f"Downloaded {filename}")

print(driver.title)
driver.quit()

Downloaded /content/drive/MyDrive/Independent_Study_2023/lawn_images/lawnimage1.jpg
Downloaded /content/drive/MyDrive/Independent_Study_2023/lawn_images/lawnimage2.jpg
Downloaded /content/drive/MyDrive/Independent_Study_2023/lawn_images/lawnimage3.jpg
Downloaded /content/drive/MyDrive/Independent_Study_2023/lawn_images/lawnimage4.jpg
Downloaded /content/drive/MyDrive/Independent_Study_2023/lawn_images/lawnimage16.jpg
Downloaded /content/drive/MyDrive/Independent_Study_2023/lawn_images/lawnimage17.jpg
Downloaded /content/drive/MyDrive/Independent_Study_2023/lawn_images/lawnimage18.jpg
Downloaded /content/drive/MyDrive/Independent_Study_2023/lawn_images/lawnimage19.jpg
Downloaded /content/drive/MyDrive/Independent_Study_2023/lawn_images/lawnimage20.jpg
Downloaded /content/drive/MyDrive/Independent_Study_2023/lawn_images/lawnimage21.jpg
Downloaded /content/drive/MyDrive/Independent_Study_2023/lawn_images/lawnimage22.jpg
Downloaded /content/drive/MyDrive/Independent_Study_2023/lawn_images/

Some helpful references used

https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

https://colab.research.google.com/drive/13oUHGDc6uy-4vJ1XnRdEcojoFoIToV75

https://colab.research.google.com/drive/13oUHGDc6uy-4vJ1XnRdEcojoFoIToV75

https://stackoverflow.com/questions/72773206/selenium-python-attributeerror-webdriver-object-has-no-attribute-find-el

https://pythoninoffice.com/fixing-attributeerror-webdriver-object-has-no-attribute-find_element_by_xpath/
